In [7]:
import os
from dotenv import load_dotenv
import vanna
from vanna.remote import VannaDefault
import mysql.connector

In [8]:
load_dotenv()

True

In [9]:
vn = VannaDefault(model='db_local', api_key=os.getenv("vanna_api_key"))

In [10]:
# Define your database connection parameters
config = {
    'user': 'root',
    'password': os.getenv("MySQL_password"),
    'host': '127.0.0.1',  # localhost
    'database': 'inventory',
    'raise_on_warnings': True
}

# Connect to the database
try:
    connection = mysql.connector.connect(**config)
    if connection.is_connected():
        print("Connected to MySQL database")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Connected to MySQL database


In [11]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [13]:
def run_mysql(mysql : str) -> pd.DataFrame:
    df = pd.read_sql_query(mysql, connection)
    return df

In [14]:
vn.run_sql = run_mysql
vn.run_sql_is_set=True

## Training

In [29]:
df_information_schema = vn.run_sql("""
    SELECT *
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'inventory'
""")

# Generate the training plan
plan = vn.get_training_plan_generic(df_information_schema)

C:\Users\jmutonyi\AppData\Local\Temp\ipykernel_3464\1837704786.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(mysql, connection)


In [30]:
for table_name in plan.get_summary():
    print(table_name)

Train on Information Schema: def.inventory item
Train on Information Schema: def.inventory item_price
Train on Information Schema: def.inventory item_stock
Train on Information Schema: def.inventory stockroom


In [31]:
vn.train(plan=plan)

### Train on DDL

In [32]:
vn.train(ddl = """
CREATE TABLE IF NOT EXIST item (
item_id INT PRIMARY KEY,
item_name VARCHAR(150),
item_description VARCHAR(150),
department_id INT,
default_price_id INT,
has_expiration INT,
concept_id INT,
creator INT,
date_created DATETIME,
changed_by INT,
date_changed DATETIME,
retired INT,
retired_by INT,
date_retired DATETIME,
retire_reason VARCHAR(50),
uuid VARCHAR(100),
has_physical_inventory INT,
default_expiration_period INT,
concept_accepted INT,
minimun_quantity INT,
buying_price DOUBLE         
)
""")

Adding ddl: 
CREATE TABLE IF NOT EXIST item (
item_id INT PRIMARY KEY,
item_name VARCHAR(150),
item_description VARCHAR(150),
department_id INT,
default_price_id INT,
has_expiration INT,
concept_id INT,
creator INT,
date_created DATETIME,
changed_by INT,
date_changed DATETIME,
retired INT,
retired_by INT,
date_retired DATETIME,
retire_reason VARCHAR(50),
uuid VARCHAR(100),
has_physical_inventory INT,
default_expiration_period INT,
concept_accepted INT,
minimun_quantity INT,
buying_price DOUBLE         
)



'288380-ddl'

### Train on Documentation

In [35]:
vn.train(documentation="This contains item inventory on a healthcare platform such as the item name, buying price, and whether it has physical inventory.")

Adding documentation....


'1812593-doc'

### Train on SQL query

In [37]:
vn.train(question="What are the item names of the items that do not have physical inventory?",
         sql = "SELECT item_name * FROM item WHERE has_physical_inventory = 0'")

'59de8a4c5423505eed60540bb7e26b7f-sql'

In [38]:
vn.get_training_data()

,id,training_data_type,question,content
0,405322-sql,sql,What are the item names of the items that do not have physical inventory?,SELECT item_name * FROM item WHERE has_physical_inventory = 0'
1,1812006-doc,documentation,None,The following columns are in the item_stock table in the def database:\n\n| | TABLE_CATALOG | TABLE_SCHEMA | TABLE_NAME | TABLE_COMMENT |\n|---:|:----------------|:---------------|:-------------|:----------------|\n| 2 | def | inventory | item_stock | |
2,1812005-doc,documentation,None,The following columns are in the item_price table in the def database:\n\n| | TABLE_CATALOG | TABLE_SCHEMA | TABLE_NAME | TABLE_COMMENT |\n|---:|:----------------|:---------------|:-------------|:----------------|\n| 1 | def | inventory | item_price | |
3,1812593-doc,documentation,None,"This contains item inventory on a healthcare platform such as the item name, buying price, and whether it has physical inventory."
4,288380-ddl,ddl,None,"\nCREATE TABLE IF NOT EXIST item (\nitem_id INT PRIMARY KEY,\nitem_name VARCHAR(150),\nitem_description VARCHAR(150),\ndepartment_id INT,\ndefault_price_id INT,\nhas_expiration INT,\nconcept_id INT,\ncreator INT,\ndate_created DATETIME,\nchanged_by INT,\ndate_changed DATETIME,\nretired INT,\nretired_by INT,\ndate_retired DATETIME,\nretire_reason VARCHAR(50),\nuuid VARCHAR(100),\nhas_physical_inventory INT,\ndefault_expiration_period INT,\nconcept_accepted INT,\nminimun_quantity INT,\nbuying_price DOUBLE \n)\n"
5,1812004-doc,documentation,None,The following columns are in the item table in the def database:\n\n| | TABLE_CATALOG | TABLE_SCHEMA | TABLE_NAME | TABLE_COMMENT |\n|---:|:----------------|:---------------|:-------------|:----------------|\n| 0 | def | inventory | item | |
6,1812007-doc,documentation,None,The following columns are in the stockroom table in the def database:\n\n| | TABLE_CATALOG | TABLE_SCHEMA | TABLE_NAME | TABLE_COMMENT |\n|---:|:----------------|:---------------|:-------------|:----------------|\n| 3 | def | inventory | stockroom | |
